In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os 
import time 

#cv2, mediapipe, and numpy are the libraries used for the camera 

mp_drawing = mp.solutions.drawing_utils #gives us drawing capabilities
mp_pose = mp.solutions.pose #pose library 

In [2]:
from gtts import gTTS
from IPython.display import Audio

def text_to_speech(text, lang='en'):
    """
    Convert text to speech and play it immediately.
    
    Parameters:
        text (str): The text to convert to speech.
        lang (str): The language of the text. Default is English ('en').
    """
    # Convert the text to speech
    tts = gTTS(text=text, lang=lang, slow=False)
    
    # Save the speech to an MP3 file
    filename = "temp_speech.mp3"
    tts.save(filename)
    
    # Play the converted file automatically
    audio = Audio(filename, autoplay=True)
    display(audio)
    
    # Optional: remove the file after playing
    os.remove(filename)


In [3]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360-angle 

    return angle

In [4]:
def calculate_back_angle(z,g):
    # Vector from right hip to right shoulder
    vec_back = np.array([g[0] - z[0], g[1] - z[1]])
    #h.s
    #s,h
    #z,g 
    
    # Calculate the angle of the back with respect to the vertical axis (negative y-axis)
    angle_back = np.arctan2(vec_back[1], vec_back[0]) * 180 / np.pi
    
    # Calculate the angle between the back and the vertical axis
    angle_vertical_axis = 90 - angle_back
    
    return angle_back, angle_vertical_axis


In [5]:
last_advice_time = 0  
advice_cooldown = 2.5

In [18]:
cap = cv2.VideoCapture(0)

counter = 0
stage = None

# Initialize pose detection
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    last_advice_time=0
    while cap.isOpened():
        ret, frame = cap.read()

        # Resize the frame to the new size
        frame = cv2.resize(frame, (800, 600))

        # Convert the BGR image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detections
        results = pose.process(image)

        # Recolor back to BGR for rendering, as opencv requires BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks 
        try:
            landmarks = results.pose_landmarks.landmark
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            shoulder2 = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow2 = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist2 = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            l_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            l_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            r_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            r_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            r_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

    

            angle = calculate_angle(shoulder, elbow, wrist)

            angle_l2 = calculate_angle(l_hip, l_knee, l_ankle)  # Corrected typo from 'l_akle' to 'l_ankle'

            angle2 = calculate_angle(shoulder2, elbow2, wrist2)

            angle_r2 = calculate_angle(r_hip, r_knee, r_ankle)

            back = calculate_back_angle(r_hip, shoulder2)



            elbow_screen = tuple(np.multiply(elbow, [800, 600]).astype(int))  # Changed resolution to match resized frame

            elbow_screen2 = tuple(np.multiply(elbow2, [800, 600]).astype(int))  # Changed resolution to match resized frame

            knee_left = tuple(np.multiply(l_knee, [800, 600]).astype(int))  # Changed resolution to match resized frame

            knee_right = tuple(np.multiply(r_knee, [800, 600]).astype(int))  # Changed resolution to match resized frame

            back_screen = tuple(np.multiply(back, [800, 600]).astype(int))




            cv2.putText(image, str(angle), 
                        elbow_screen, 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, 
                        (255, 255, 255), 
                        2, 
                        cv2.LINE_AA)

            cv2.putText(image, str(angle), 
                        back_screen, 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, 
                        (255, 255, 255), 
                        2, 
                        cv2.LINE_AA)

            cv2.putText(image, str(angle2), 
                        elbow_screen2, 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, 
                        (255, 255, 255), 
                        2, 
                        cv2.LINE_AA)

            cv2.putText(image, str(angle_l2), 
                        knee_left, 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, 
                        (255, 255, 255), 
                        2, 
                        cv2.LINE_AA)

            cv2.putText(image, str(angle_r2), 
                        knee_right, 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, 
                        (255, 255, 255), 
                        2, 
                        cv2.LINE_AA)

            
            
        

                           # Inside the loop where you check squat conditions:
            current_time = time.time()  # Always update the current time at the start of the loop
            
            # Detect when the user starts going down
            if angle_r2 > 170 and stage != "down":
                stage = "down"
                tts_lower_triggered = False  # Reset advice flag when starting a new squat
            
            # Check if user is not low enough in the squat
            elif 90 < angle_r2 <= 130 and stage == "down" and not tts_lower_triggered:
                text_to_speech('squat is good, but go lower with your legs, if you cannot, maybe decrease weight')
                tts_lower_triggered = True  # Prevent repeating advice in the same squat
            
            # Detect when the user comes up from a squat
            elif angle_r2 <= 90:
                if stage == "down":  # Only count as a completed squat if the user was down before
                    counter += 1
                    print(counter)
                    text_to_speech('squat done!')
                stage = "up"
        
        
        except Exception as e:
            print(e)  # This will help to understand what goes wrong in the try block

        # Render and draw landmarks into the image
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                                
        cv2.imshow('Mediapipe Feed', image)

        # Break the loop when 'q' is pressed
        if cv2.waitKey(50) & 0xFF == ord('q'):
            break

# Release resources
cap.release()
cv2.destroyAllWindows()

'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
1


'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
2


3


'NoneType' object has no attribute 'landmark'
4


'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute